In [1]:
# BLOQUE 1: IMPORTACIONES Y CONFIGURACIÓN

import argparse # 'argparse' nos ayuda a manejar opciones, aunque aquí usaremos variables directas.
import os # 'os' nos permite interactuar con el sistema operativo (crear carpetas, leer archivos).
import numpy as np # 'numpy' es la calculadora científica fundamental de Python (maneja matrices de números).
import math # 'math' nos da funciones matemáticas básicas.
import torchvision.transforms as transforms # 'torchvision.transforms' contiene herramientas para editar imágenes (rotar, cambiar tamaño, normalizar).
from torchvision.utils import save_image# 'save_image' es una función específica para guardar nuestros resultados como .png o .jpg.
from torch.utils.data import DataLoader # 'DataLoader' es el encargado de cargar los datos en lotes (paquetes) para no llenar la memoria RAM.
from torchvision import datasets # 'datasets' tiene bases de datos famosas listas para usar (como MNIST).
from torch.autograd import Variable # 'Variable' es un envoltorio antiguo de PyTorch, pero útil en versiones legacy para decir "esto se va a entrenar".
import torch.nn as nn # 'torch.nn' contiene los bloques de construcción de redes neuronales (capas densas, convoluciones).
import torch.nn.functional as F # 'torch.nn.functional' contiene funciones sin estado (como activaciones o pérdidas).
import torch # 'torch' es la librería principal (el cerebro de todo).


# Creamos una carpeta llamada "images" para guardar el progreso del generador.
# exist_ok=True evita que el código falle si la carpeta ya existe.
os.makedirs("images", exist_ok=True)

# --- HIPERPARÁMETROS (Las perillas que ajustamos) ---
n_epochs = 50        # Número de veces que el modelo verá todos los datos (Ciclos de entrenamiento).
batch_size = 64      # Cuántas imágenes procesa la IA al mismo tiempo (Lotes).
lr = 0.0002          # Learning Rate: Qué tan rápido aprende (muy alto = inestable, muy bajo = lento).
b1 = 0.5             # Un parámetro interno del optimizador Adam (maneja el impulso).
b2 = 0.999           # Otro parámetro interno de Adam.
n_cpu = 8            # Cuántos núcleos del procesador usar para cargar datos.
latent_dim = 100     # Tamaño del "ruido" inicial. Es la semilla de la imaginación de la IA.
img_size = 28        # Tamaño de la imagen (MNIST son números de 28x28 píxeles).
channels = 1         # Canales de color: 1 = Blanco y Negro, 3 = RGB.
sample_interval = 400 # Cada cuántos pasos guardamos una imagen de prueba.

# Definimos el tamaño exacto de la imagen multiplicando sus dimensiones.
# 1 * 28 * 28 = 784 píxeles totales por imagen.
img_shape = (channels, img_size, img_size)

# Verificamos si Google Colab nos prestó una GPU (Tarjeta Gráfica).
# Si 'cuda' es True, el entrenamiento será rapidísimo. Si es False, será lento.
cuda = True if torch.cuda.is_available() else False
print(f"¿Usando GPU?: {cuda}")

¿Usando GPU?: True


In [2]:
# BLOQUE 2: DEFINICIÓN DEL GENERADOR

class Generator(nn.Module):
    def __init__(self):
        # Inicializamos la clase padre (necesario en Python para heredar funciones).
        super(Generator, self).__init__()

        # Definimos las capas de la red como una lista de pasos secuenciales.
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)] # Capa lineal: conecta todas las entradas con las salidas.
            if normalize:
                # BatchNormalization: Reajusta los valores matemáticos para que el aprendizaje no se descontrole.
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            # LeakyReLU: Una función de activación que deja pasar señales negativas pequeñas (evita neuronas muertas).
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        # Construimos el modelo apilando bloques.
        # La red va creciendo: Entra ruido (100) -> 128 -> 256 -> 512 -> 1024 neuronas.
        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            # Capa final: Convierte las 1024 neuronas en los 784 píxeles de la imagen.
            nn.Linear(1024, int(np.prod(img_shape))),
            # Tanh: Aplasta los valores para que estén entre -1 y 1 (formato estándar de imágenes en IA).
            nn.Tanh()
        )

    # La función 'forward' define cómo pasan los datos por la red.
    def forward(self, z):
        # Pasamos el ruido 'z' por el modelo definido arriba.
        img = self.model(z)
        # Reorganizamos la lista de números (784) en forma de imagen (1 canal, 28 alto, 28 ancho).
        img = img.view(img.size(0), *img_shape)
        return img

In [3]:
# BLOQUE 3: DEFINICIÓN DEL DISCRIMINADOR

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            # Capa 1: Recibe la imagen aplanada (784 píxeles) y la comprime a 512 neuronas.
            nn.Linear(int(np.prod(img_shape)), 512),
            # LeakyReLU: Activación para aprender patrones complejos.
            nn.LeakyReLU(0.2, inplace=True),

            # Capa 2: De 512 a 256 neuronas.
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),

            # Capa Final: De 256 a 1 sola neurona.
            # ¿Por qué 1? Porque la respuesta es binaria: 0 (Falso) o 1 (Real).
            nn.Linear(256, 1),
            # Sigmoid: Convierte el número final en una probabilidad entre 0% y 100%.
            nn.Sigmoid(),
        )

    def forward(self, img):
        # Aplanamos la imagen: de (1, 28, 28) a una fila larga de (784).
        img_flat = img.view(img.size(0), -1)
        # Pasamos la imagen plana por el modelo para obtener el veredicto (validez).
        validity = self.model(img_flat)
        return validity

In [4]:
# BLOQUE 4: INICIALIZACIÓN

# Calculamos la "Pérdida" (Loss). Usamos BCELoss (Binary Cross Entropy).
# Es la fórmula matemática que mide qué tan mal se equivocó el modelo (Policía vs Ladrón).
adversarial_loss = torch.nn.BCELoss()

# Instanciamos (creamos) los dos modelos.
generator = Generator()
discriminator = Discriminator()

# Si tenemos GPU (cuda), movemos los modelos y la función de pérdida a la tarjeta gráfica.
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# --- CARGA DE DATOS ---
# Configuramos el DataLoader.
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist", # Carpeta donde se guardarán los datos.
        train=True,         # Queremos los datos de entrenamiento.
        download=True,      # Descargarlos si no existen.
        transform=transforms.Compose(
            # Redimensionamos a 28x28 (por seguridad).
            [transforms.Resize(img_size),
             # Convertimos la imagen a Tensor (formato numérico de PyTorch).
             transforms.ToTensor(),
             # Normalizamos: convertimos los píxeles de [0,1] a [-1,1] para que coincida con Tanh.
             transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size, # Tamaño del paquete.
    shuffle=True,          # Barajamos los datos para que la IA no memorice el orden.
)

# Optimizadores: Son los algoritmos que actualizan los "pesos" (conocimiento) de la red.
# Usamos Adam, que es muy popular y eficiente.
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

# Definimos el tipo de dato Tensor según si usamos GPU o CPU.
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 502kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.62MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.56MB/s]


In [5]:
# BLOQUE 5: ENTRENAMIENTO

print("Iniciando entrenamiento... (Esto puede tardar unos minutos)")

# Ciclo principal: Repetimos por el número de 'epochs'.
for epoch in range(n_epochs):
    # Ciclo interno: Repetimos por cada lote de imágenes en el dataset.
    for i, (imgs, _) in enumerate(dataloader):

        # --- PREPARAR ETIQUETAS ---
        # Creamos etiquetas de "Verdad" (1.0) para decir que una imagen es Real.
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        # Creamos etiquetas de "Mentira" (0.0) para decir que una imagen es Falsa.
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Convertimos las imágenes reales al formato adecuado (Tensor).
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  ENTRENAR GENERADOR (El Falsificador)
        # -----------------
        optimizer_G.zero_grad() # Limpiamos los gradientes (errores) anteriores.

        # Generamos ruido aleatorio (la "semilla" creativa).
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # El Generador crea imágenes falsas a partir del ruido.
        gen_imgs = generator(z)

        # Calculamos qué tan bien engañó al discriminador.
        # Queremos que el discriminador diga que estas imágenes son Válidas (1.0).
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        # Backpropagation: Calculamos cómo ajustar los pesos para mejorar el engaño.
        g_loss.backward()
        # Actualizamos el cerebro del Generador.
        optimizer_G.step()

        # ---------------------
        #  ENTRENAR DISCRIMINADOR (El Policía)
        # ---------------------
        optimizer_D.zero_grad() # Limpiamos gradientes.

        # Medimos qué tan bien clasifica las imágenes REALES (debe decir 1.0).
        real_loss = adversarial_loss(discriminator(real_imgs), valid)

        # Medimos qué tan bien clasifica las imágenes FALSAS (debe decir 0.0).
        # Usamos .detach() para no afectar al generador en este paso.
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)

        # Promediamos ambas pérdidas (detectar real y detectar falso).
        d_loss = (real_loss + fake_loss) / 2

        # Backpropagation del policía.
        d_loss.backward()
        # Actualizamos el cerebro del Discriminador.
        optimizer_D.step()

        # --- REPORTE DE PROGRESO ---
        # Imprimimos el estado cada cierto tiempo para saber que no se congeló.
        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            print(f"[Epoch {epoch}/{n_epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")
            # Guardamos una muestra de las imágenes generadas para ver cómo mejora.
            save_image(gen_imgs.data[:25], f"images/{batches_done}.png", nrow=5, normalize=True)

# --- GUARDADO FINAL ---
# Al terminar todas las épocas, guardamos el "cerebro" entrenado en un archivo.
# Este archivo .pth es el que usaremos en la App de Gradio.
torch.save(generator.state_dict(), 'generador_mnist.pth')
print("¡Entrenamiento completado! Modelo guardado como 'generador_mnist.pth'")

Iniciando entrenamiento... (Esto puede tardar unos minutos)


/tmp/ipython-input-3665318708.py:12: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:78.)
  valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)


[Epoch 0/50] [Batch 0/938] [D loss: 0.7065] [G loss: 0.6851]
[Epoch 0/50] [Batch 400/938] [D loss: 0.4828] [G loss: 0.9234]
[Epoch 0/50] [Batch 800/938] [D loss: 0.4045] [G loss: 1.0325]
[Epoch 1/50] [Batch 262/938] [D loss: 0.4830] [G loss: 1.0975]
[Epoch 1/50] [Batch 662/938] [D loss: 0.5366] [G loss: 1.9022]
[Epoch 2/50] [Batch 124/938] [D loss: 0.4550] [G loss: 1.0367]
[Epoch 2/50] [Batch 524/938] [D loss: 0.2896] [G loss: 2.3139]
[Epoch 2/50] [Batch 924/938] [D loss: 0.3697] [G loss: 1.9196]
[Epoch 3/50] [Batch 386/938] [D loss: 0.2918] [G loss: 2.5375]
[Epoch 3/50] [Batch 786/938] [D loss: 0.5577] [G loss: 3.2851]
[Epoch 4/50] [Batch 248/938] [D loss: 0.1696] [G loss: 2.0513]
[Epoch 4/50] [Batch 648/938] [D loss: 0.4110] [G loss: 2.7737]
[Epoch 5/50] [Batch 110/938] [D loss: 0.1541] [G loss: 2.2486]
[Epoch 5/50] [Batch 510/938] [D loss: 0.3268] [G loss: 1.3346]
[Epoch 5/50] [Batch 910/938] [D loss: 0.2659] [G loss: 1.3640]
[Epoch 6/50] [Batch 372/938] [D loss: 0.3486] [G loss: 1.

In [7]:
# BLOQUE 6

!pip install gradio

import gradio as gr
import torch
import numpy as np
import torch.nn.functional as F  # Importamos funciones para redimensionar

# 1. Cargar el modelo (Igual que antes)
def cargar_modelo():
    modelo = Generator()
    # Asegúrate de que el archivo .pth exista en la carpeta
    if torch.cuda.is_available():
        modelo.load_state_dict(torch.load('generador_mnist.pth'))
        modelo.cuda()
    else:
        modelo.load_state_dict(torch.load('generador_mnist.pth', map_location=torch.device('cpu')))
    modelo.eval()
    return modelo

generador_entrenado = cargar_modelo()

# 2. Función de Generación con "LUPA" (Zoom)
def generar_digito(semilla):
    # Convertir semilla
    seed = int(semilla)
    torch.manual_seed(seed)

    # Generar ruido y procesar en el dispositivo correcto (CPU o GPU)
    device = next(generador_entrenado.parameters()).device
    z = torch.randn(1, 100).to(device)

    with torch.no_grad():
        # Generamos la imagen original (Tamaño 28x28)
        img_tensor = generador_entrenado(z)

        # --- AQUÍ ESTÁ EL TRUCO DEL ZOOM ---
        # Usamos interpolate para agrandar la imagen 10 veces (de 28 a 280 px)
        # mode='nearest' mantiene los bordes definidos (estilo pixel art) para que no se vea borroso.
        img_tensor = F.interpolate(img_tensor, scale_factor=10, mode='nearest')

    # Procesar para mostrar
    img_array = img_tensor.cpu().view(280, 280).numpy() # Ahora es 280x280
    img_array = (img_array + 1) / 2

    return img_array

# 3. Interfaz
interfaz = gr.Interface(
    fn=generar_digito,
    inputs=[
        gr.Slider(minimum=0, maximum=1000, value=42, step=1, label="Código de Variación (Semilla)")
    ],
    outputs=gr.Image(label="Dígito Generado (Zoom 10x)", type="numpy"), # Especificamos salida de imagen
    title="Generador de Datos Sintéticos MNIST (Alta Visibilidad)",
    description="Herramienta de IA Generativa. Mueve el slider para generar dígitos manuscritos sintéticos. Se ha aplicado un escalado 10x para facilitar la visualización.",
    theme="default"
)

interfaz.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://41ee4ca2c0e41a1677.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Created dataset file at: .gradio/flagged/dataset2.csv
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://41ee4ca2c0e41a1677.gradio.live
